In [1]:
using DataFrames

using Tables

In [2]:
n = 10^2

ro_x_co_x_an = DataFrame(
    "In"=>rand(1:9, n),
    "Fl"=>rand(1.0:9, n),
    "Ch"=>rand('a':'z', n),
    "St4"=>[join(rand('a':'z', 4)) for _ in 1:n],
    "St8"=>[join(rand('a':'z', 8)) for _ in 1:n],
)

;

What is the best way to read one DataFrame column?

In [3]:
# Everything in `()` is evaluated before the trial, making this a bad benchmark.
@btime $(ro_x_co_x_an[!, "St4"])

# Evaluate the global mutable variable (interpolate) so that the compiler knows more.

@btime ro_x_co_x_an[!, "St4"]

@btime $ro_x_co_x_an[!, "St4"]

# `view` allocates and is slower (a surprise).
@btime view($ro_x_co_x_an, !, "St4")

;

  2.125 ns (0 allocations: 0 bytes)
  121.050 ns (0 allocations: 0 bytes)
  97.427 ns (0 allocations: 0 bytes)
  189.233 ns (1 allocation: 48 bytes)


What is the best way to read two DataFrame columns?

In [4]:
# `zip` DataFrame.
function zi(da)
    
    for (a, b) in zip(da[!, "St4"], da[!, "St8"])
        
    end
    
end

# `eachrow` DataFrame.
function ea(da)
    
    for (a, b) in eachrow(da[!, ["St4", "St8"]])
        
    end
    
end

# `zip` method allocates more but is faster.

@btime zi($ro_x_co_x_an)

@btime ea($ro_x_co_x_an)

;

  15.000 μs (504 allocations: 17.31 KiB)
  21.208 μs (220 allocations: 7.98 KiB)


In [5]:
# Make `Matrix` and `zip`.
function mzi(da)
    
    ma = Matrix(da)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

# Make `Matrix` and `eachrow`.
function mea(da)
    
    ma = Matrix(da)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

# `Matrix`ing is slower...

@btime mzi($ro_x_co_x_an)

@btime mea($ro_x_co_x_an)

;

  25.791 μs (813 allocations: 31.20 KiB)
  32.042 μs (910 allocations: 31.20 KiB)


Does breaking the functions into smaller ones improve the performance?

In [6]:
# The part of the code to make a `Matrix`.
function mt(da)
    
    Matrix(da)
    
end

@btime mt($ro_x_co_x_an)

;

  3.083 μs (105 allocations: 5.86 KiB)


In [7]:
# The part of the code to `zip`.
function mazi(ma)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

# The part of the code to `eachrow`.
function maea(ma)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

ma = Matrix(ro_x_co_x_an)

@btime mazi($ma)

@btime maea($ma)

;

  322.231 ns (2 allocations: 1.75 KiB)
  361.310 ns (2 allocations: 1.84 KiB)


Sum of the parts (3 us + 0.3 us = 3.3 us) were less than their combined (20 us).

Does substituting these parts improve the performance?

In [8]:
# Use a function to barrier the code that makes a `Matirx`.

function mt_mzi(da)
    
    ma = mt(da)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

function mt_mea(da)
    
    ma = mt(da)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

@btime mt_mzi($ro_x_co_x_an)

@btime mt_mea($ro_x_co_x_an)

;

  25.709 μs (813 allocations: 31.20 KiB)
  31.958 μs (910 allocations: 31.20 KiB)


In [9]:
# Using a function to barrier the code that loops a `Matirx`.

function _mazi(da)
    
    ma = Matrix(da)
    
    mazi(ma)
    
end

function _maea(da)
    
    ma = Matrix(da)
    
    maea(ma)
    
end

@btime _mazi($ro_x_co_x_an)

@btime _maea($ro_x_co_x_an)

;

  3.448 μs (107 allocations: 7.61 KiB)
  3.516 μs (107 allocations: 7.70 KiB)


In [10]:
# Barrier both.

function mt_mazi(da)
    
    ma = mt(da)
    
    mazi(ma)
    
end

function mt_maea(da)
    
    ma = mt(da)
    
    maea(ma)
    
end

@btime mt_mazi($ro_x_co_x_an)

@btime mt_maea($ro_x_co_x_an)

;

  3.438 μs (107 allocations: 7.61 KiB)
  3.521 μs (107 allocations: 7.70 KiB)


The compiler optimizes code at function boundaries.
Use multiple smaller functions! 

But what is really happening here is that `DataFrame` columns can be of any type.
`DataFrame` is not parameterized but its `field`s (columns) are.
So, the columns can change during run time and the compiler can not know about them or objects (like `Matirx`) derived from them.

At the function barrier, the compiler knows that the function gets a `DataFrame`-derived `Matrix` (but not its type) and dispatches to `Matirx{Any}`.
The compiler does so because `Matirx` is not a concrete type but `Matrix{Any}` is.

In the slower code, if we helped the compiler know that the `Matrix` is `Matrix{Any}`, the compiler does less work and the performance is even better than using the function barrier.

In [11]:
# Make `Matrix{Any}` and `zip`.
function mzi(da)
    
    ma = Matrix{Any}(da)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

# Make `Matrix{Any}` and `eachrow`.
function mea(da)
    
    ma = Matrix{Any}(da)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

# `Matrix{Any}`ing is the fastest.

@btime mzi($ro_x_co_x_an)

@btime mea($ro_x_co_x_an)

;

  1.817 μs (104 allocations: 7.47 KiB)
  1.846 μs (104 allocations: 7.56 KiB)


https://discourse.julialang.org/t/matrix-vs-matrix-any/90911

`Matrix(::DataFrame)` makes `Matrix` not `Matrix{Any}`.
The compiler can not know the type at compile time and must figure it out at run time.
All subsequent operations using this object must be dynamically dispatched.
Dynamic dispatch is like a big `if-elseif` code, which has many branches and prevents the processor from pre-computing.

Typing the object with an assertion helps the compiler know the type at compile time and do optimizations.
The compiler inlines omits, and reorders code as fit.
We want the compiler to do more work at compile time and less at run time.
We want the execution to be fast.
Withtout the compiler doing work at the compile time, Julia becomes just an interpreter with high latency.

`Any` is an abstract type.
`Matrix{Any}` is also problematic for the same reason above.
The compiler must dynamically dispatch operations for it at run time.